# Project Group - 5

Members: Ahmad Nabil Maulana, Daan Michel, Gijs Aben, Philine Cremers

Student numbers: 5943442, 4684559 ,    4713656   ,     5036534

# Research Objective

*Requires data modeling and quantitative research in Transport, Infrastructure & Logistics*

The research objective is to analyse the Impact of Gross Domestic Product pet capita (GDP) on the number of air traffic movements within a country. 

# Contribution Statement

*Be specific. Some of the tasks can be coding (expect everyone to do this), background research, conceptualisation, visualisation, data analysis, data modelling*

**Author 1**: Ahmad Nabil Maulana : Creating the repository,

**Author 2**: Daan Michel

**Author 3**: Philine Cremers

**Author 4**: Gijs Aben

# Background and Context

For this project, we will analyze the relationship between a country's GDP per capita and its traffic mobility. GDP measures the monetary value of final goods and services produced within a country's borders during a specific period, such as a quarter or a year. We aim to determine whether countries with higher GDPs exhibit greater air traffic mobility compared to those with lower GDPs, or if there is no discernible correlation

For the flight data we use all air traffic movement to and from all the airports in a country. The only aircraft that are used are comercial aircraft and cargo aircraft. Private aircraft are not considered.

As the first step, we need to import the necessary libraries.

In [2]:
# this can be changed
import pandas as pd
from pathlib import Path
import numpy as np
import math
import scipy
from scipy.signal import find_peaks
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd

# Part I - Data Import

First, we're going to import and combine dataframes from the two datas that we will be using:

* Countries' GDP data from The World Bank.
* Countries' air traffic mobility data from OpenSky.

# Part II - Data Processing